# Bulk Grocery Sales Analysis

## Sales Summary

### Total Sales, Transactions, Customers, Products

In [8]:
SELECT 
    SUM(l.SaleAmount) AS Sales,
    COUNT(DISTINCT o.OrderID) AS TransactionCount,
    COUNT(DISTINCT o.CustomerID) AS KnownCustomerCount,
    COUNT(DISTINCT l.ProductID) AS UniqueProductCount,
    COUNT(DISTINCT l.SalesOrderLineID) AS TransactionLineCount

FROM SalesOrderLines AS l
    JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 



(1 row affected)

Total execution time: 00:00:00.165

Sales,TransactionCount,KnownCustomerCount,UniqueProductCount,TransactionLineCount
1308799.38,41450,3932,1006,223151


### Annual Sales Volume
#### Total Annual Sales

In [11]:
SELECT 
    YEAR(o.OrderDate) AS Year, 
    SUM(l.SaleAmount) AS Sales, 
    COUNT(DISTINCT o.OrderID) As TransactionCount,
    COUNT(DISTINCT o.CustomerID) AS KnownCustomerCount,
    COUNT(DISTINCT l.ProductID) AS UniqueProductCount

FROM SalesOrderLines AS l
    JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 

GROUP BY YEAR(o.OrderDate)
ORDER BY YEAR(o.OrderDate) 

(3 rows affected)

Total execution time: 00:00:00.181

Year,Sales,TransactionCount,KnownCustomerCount,UniqueProductCount
2019,277348.76,11776,1704,673
2020,494122.18,14648,1769,635
2021,537328.44,15026,2137,582


### Monthly Sales Trend
#### Total Monthly Sales and Average Daily Sales by Month

In [13]:
SELECT 
    EOMONTH(o.OrderDate) AS TransactionMonth, --Use the last day of the month to represent the Month. Since all data is historical do not need to account for midmonth data
    ROUND(SUM(l.SaleAmount),2) AS Sales, --Sum the Monthly Sales
    ROUND(SUM(l.SaleAmount)/DAY(EOMONTH(o.OrderDate)),2) AS AvgDailySales, --Total Montly Sales / Number of Days in Month
    COUNT(DISTINCT o.OrderID) AS TransactionCount

FROM SalesOrderLines AS l
    JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 

GROUP BY EOMONTH(o.OrderDate)
ORDER BY EOMONTH(o.OrderDate)   

(35 rows affected)

Total execution time: 00:00:00.144

TransactionMonth,Sales,AvgDailySales,TransactionCount
2019-02-28,14964.79,534.460000,749
2019-03-31,20949.45,675.790000,1003
2019-04-30,17287.15,576.240000,899
2019-05-31,24424.98,787.900000,1184
2019-06-30,27454.37,915.150000,1137
2019-07-31,27423.94,884.640000,1120
2019-08-31,26306.96,848.610000,1095
2019-09-30,28842.81,961.430000,1109
2019-10-31,29368.16,947.360000,1135
2019-11-30,29010.85,967.030000,1153


#### Top 5 Sales Months

In [20]:
SELECT TOP(5)
    EOMONTH(o.OrderDate) AS TransactionMonth, --Use the last day of the month to represent the Month. Since all data is historical do not need to account for midmonth data
    ROUND(SUM(l.SaleAmount),2) AS MonthlySales, --Sum the Monthly Sales
    ROUND(SUM(l.SaleAmount)/DAY(EOMONTH(o.OrderDate)),2) AS AvgDailySales -- Total Monthly Sales / Number of Days in month

FROM SalesOrderLines AS l
    JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 

GROUP BY EOMONTH(o.OrderDate) --Group by Month
ORDER BY MonthlySales DESC     

(5 rows affected)

Total execution time: 00:00:00.120

TransactionMonth,MonthlySales,AvgDailySales
2021-01-31,55312.48,1784.270000
2021-02-28,54057.65,1930.630000
2020-12-31,53039.45,1710.950000
2021-04-30,52109.81,1736.990000
2021-03-31,51043.98,1646.580000


#### Monthly Sales - MoM, YoY % Change - Using Window LAG function and CTEs


In [21]:
--CTE to create a table of Total Sales by Month 
WITH MonthlySales AS(
SELECT 
    EOMONTH(o.OrderDate) AS TransactionMonth, --Use the last day of the month to represent the Month. Since all data is historical do not need to account for midmonth data
    ROUND(SUM(l.SaleAmount),2) AS Sales, --Sum the Monthly Sales
    ROUND(SUM(l.SaleAmount)/DAY(EOMONTH(o.OrderDate)),2) AS AvgDailySales  --Monthly Sales / number of days in the month
FROM SalesOrderLines AS l
    JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 
GROUP BY EOMONTH(o.OrderDate)
)

-- Compare Each Month to the Prior Month, and the Prior Year
SELECT  
    TransactionMonth,
    Sales,
    
    -- Monthly Sales Percent Change from Prior Month = Current month sales / Prior month sales - 1 
    ROUND(Sales/ 
        (LAG(Sales) -- use the LAG Window function with no offset, which defaults to the prior record (offset of 1)
            OVER ( ORDER BY TransactionMonth))-1,4) -- subtract 1 from the ratio of current/prior month sales to calculate the percentage change
            AS SalesPctChgPM, 
    
    
    -- Monthly Sales % Change from Same Month Prior Year = Current Month/ Lag 12 month Sales -1
    ROUND(Sales/ 
    (LAG(Sales,12) -- Use the Lag Function with offset of 12 to retrive the prior year sales (assuming continuous sales reported in all months)
        OVER (ORDER BY TransactionMonth))-1,4) -- subtract 1 from the ratio and round to 4 decimals
        AS SalesPctChgPY, 
    
    AvgDailySales,
    
    -- Avg Daily Sales % change from Prior Month = Avg Daily Sales Current Month / ADS Prior Month-1 
    ROUND((AvgDailySales/
    LAG(AvgDailySales) OVER(ORDER BY TransactionMonth))-1,4) AS AvgDailyPctChgPM,

    -- Avg Daily Sales % change from Prior Year = Avg Daily Sales Current Month / ADS Prior Year-1 
    -- Assumes sales in all consecutive months
    ROUND((AvgDailySales/
    LAG(AvgDailySales,12) OVER(ORDER BY TransactionMonth))-1,4) AS AvgDailyPctChgPY

FROM MonthlySales

(35 rows affected)

Total execution time: 00:00:00.120

TransactionMonth,Sales,SalesPctChgPM,SalesPctChgPY,AvgDailySales,AvgDailyPctChgPM,AvgDailyPctChgPY
2019-02-28,14964.79,NULL,NULL,534.460000,NULL,NULL
2019-03-31,20949.45,0.399900,NULL,675.790000,0.264400,NULL
2019-04-30,17287.15,-0.174800,NULL,576.240000,-0.147300,NULL
2019-05-31,24424.98,0.412900,NULL,787.900000,0.367300,NULL
2019-06-30,27454.37,0.124000,NULL,915.150000,0.161500,NULL
2019-07-31,27423.94,-0.001100,NULL,884.640000,-0.033300,NULL
2019-08-31,26306.96,-0.040700,NULL,848.610000,-0.040700,NULL
2019-09-30,28842.81,0.096400,NULL,961.430000,0.132900,NULL
2019-10-31,29368.16,0.018200,NULL,947.360000,-0.014600,NULL
2019-11-30,29010.85,-0.012200,NULL,967.030000,0.020800,NULL


#### Three Month Moving Average Monthly Sales - Using Window Function

In [85]:
--CTE to create a table of Total Sales by Month and by Product Category
WITH MonthlySales AS(
SELECT 
    EOMONTH(o.OrderDate) AS TransactionMonth, --Use the last day of the month to represent the Month. Since all data is historical do not need to account for midmonth data
    ROUND(SUM(l.SaleAmount),2) AS Sales, --Sum the Monthly Sales
    ROUND(SUM(l.SaleAmount)/DAY(EOMONTH(o.OrderDate)),2) AS AvgDailySales
FROM SalesOrderLines AS l
    JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 
GROUP BY EOMONTH(o.OrderDate)
)

-- Caculate a rolling three month average of sales by category
SELECT  
    TransactionMonth,
    SUM(Sales) TotalSales,    
    AVG(SUM(Sales)) OVER(
        ORDER BY TransactionMonth -- Order the records by transaction month, ensuring the average is calculated chronologically
        ROWS BETWEEN 2 PRECEDING and CURRENT ROW -- specifies the range of rows to include in the average as 2 preceeding and current row
    ) AS MovingThreeMonthAvgSales,

FROM MonthlySales
GROUP BY TransactionMonth
ORDER BY TransactionMonth

(35 rows affected)

Total execution time: 00:00:00.139

TransactionMonth,TotalSales,MovingThreeMonthAvgSales
2019-02-28,14964.79,14964.790000
2019-03-31,20949.45,17957.120000
2019-04-30,17287.15,17733.796666
2019-05-31,24424.98,20887.193333
2019-06-30,27454.37,23055.500000
2019-07-31,27423.94,26434.430000
2019-08-31,26306.96,27061.756666
2019-09-30,28842.81,27524.570000
2019-10-31,29368.16,28172.643333
2019-11-30,29010.85,29073.940000


#### Three Month Moving Average Monthly Sales by Product Category - Using Window Function with Partition

Trailing Three Month Average Sales by Product Category

In [13]:
--CTE to create a table of Total Sales by Month and by Product Category
WITH MonthlySales AS(
SELECT 
    c.Category AS Category,
    EOMONTH(o.OrderDate) AS TransactionMonth, --Use the last day of the month to represent the Month. Since all data is historical do not need to account for midmonth data
    ROUND(SUM(l.SaleAmount),2) AS Sales, --Sum the Monthly Sales
    ROUND(SUM(l.SaleAmount)/DAY(EOMONTH(o.OrderDate)),2) AS AvgDailySales
FROM SalesOrderLines AS l
    JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 
    JOIN DimProduct AS p
    ON l.ProductID = p.ProductID
    JOIN DimSubcategory AS sc
    ON p.SubcategoryID = sc.SubcategoryID
    JOIN DimCategory AS c
    ON sc.CategoryID = c.CategoryID
GROUP BY Category, EOMONTH(o.OrderDate)
)

-- Caculate a rolling three month average of sales by category
SELECT  
    Category,
    TransactionMonth,
    SUM(Sales) TotalSales,    
    AVG(SUM(Sales)) OVER(
        PARTITION BY Category  -- Divide the data by Category - the avg will be calcuated separately for each category
        ORDER BY TransactionMonth -- Order the records by transaction month, ensuring the average is calculated chronologically
        ROWS BETWEEN 2 PRECEDING and CURRENT ROW -- specifies the range of rows to include in the average as 2 preceeding and current row
    ) AS MovingThreeMonthAvgSales

FROM MonthlySales
GROUP BY
    Category,
    TransactionMonth
ORDER BY Category, TransactionMonth


(337 rows affected)

Total execution time: 00:00:00.179

Category,TransactionMonth,TotalSales,MovingThreeMonthAvgSales
Accessories,2019-02-28,742.06,742.060000
Accessories,2019-03-31,752.48,747.270000
Accessories,2019-04-30,903.54,799.360000
Accessories,2019-05-31,1094.00,916.673333
Accessories,2019-06-30,1100.73,1032.756666
Accessories,2019-07-31,1003.34,1066.023333
Accessories,2019-08-31,940.90,1014.990000
Accessories,2019-09-30,1073.68,1005.973333
Accessories,2019-10-31,1048.93,1021.170000
Accessories,2019-11-30,978.33,1033.646666


## Product Analysis

#### Top Selling Product Categories and % of Total Revenue

In [23]:
SELECT TOP (10)
    c.Category,
    SUM(l.SaleAmount) AS TotalSales,
    SUM(l.SaleAmount)/
        (SELECT SUM(l.SaleAmount)
            FROM SalesOrderLines AS l) AS PctOfTotal
        
FROM SalesOrderLines AS l
    JOIN DimProduct AS p
    ON l.ProductID = p.ProductID
    JOIN DimSubcategory AS sc
    ON p.SubcategoryID = sc.SubcategoryID
    JOIN DimCategory AS c
    ON sc.CategoryID = c.CategoryID

GROUP BY c.Category
ORDER BY SUM(l.SaleAmount) DESC

(10 rows affected)

Total execution time: 00:00:00.588

Category,TotalSales,PctOfTotal
Grocery,605584.67,0.462702
Local Love,247621.71,0.189197
Body Care,175967.19,0.134449
Cleaning Supply,98378.00,0.075166
Whls_Grocery,54745.46,0.041828
Accessories,48086.01,0.036740
Containers,32241.63,0.024634
Fresh Produce,31740.42,0.024251
Grab & Go,8398.09,0.006416
Gift Card,5014.71,0.003831


#### Top Selling Product Subcategories and % of Total Revenue

In [25]:
SELECT TOP (10)
    sc.Subcategory,
    c.Category,
    SUM(l.SaleAmount) AS TotalSales,
    SUM(l.SaleAmount)/
        (SELECT SUM(l.SaleAmount)
            FROM SalesOrderLines AS l) AS PctOfTotal

FROM SalesOrderLines AS l
    JOIN DimProduct AS p
    ON l.ProductID = p.ProductID
    JOIN DimSubcategory AS sc
    ON p.SubcategoryID = sc.SubcategoryID
    JOIN DimCategory AS c
    ON sc.CategoryID = c.CategoryID

GROUP BY sc.Subcategory, c.Category
ORDER BY SUM(l.SaleAmount) DESC

(10 rows affected)

Total execution time: 00:00:00.160

Subcategory,Category,TotalSales,PctOfTotal
Nuts,Grocery,98596.61,0.075333
Body Cleansing,Body Care,70617.06,0.053955
Seeds And Grains,Grocery,67660.29,0.051696
Oils,Grocery,66665.40,0.050936
Laundry Cleaners,Cleaning Supply,54217.30,0.041425
Dried Fruits,Grocery,42389.19,0.032387
Sweeteners,Grocery,42159.78,0.032212
Kombucha,Local Love,41918.87,0.032028
Hair Care,Body Care,40704.96,0.031100
Dairy,Grocery,39807.90,0.030415


#### Top Selling Products and % of Total Revenue

In [26]:
SELECT TOP(20)
    p.ProductName,
    sc.Subcategory,
    c.Category,
    SUM(l.SaleAmount) AS TotalSales,
    
    SUM(l.SaleAmount)/
        (SELECT SUM(l.SaleAmount)
            FROM SalesOrderLines AS l) AS PctOfTotal


FROM SalesOrderLines AS l
    JOIN DimProduct AS p
    ON l.ProductID = p.ProductID
    JOIN DimSubcategory AS sc
    ON p.SubcategoryID = sc.SubcategoryID
    JOIN DimCategory AS c
    ON sc.CategoryID = c.CategoryID

GROUP BY p.ProductName, sc.Subcategory, c.Category
ORDER BY SUM(l.SaleAmount) DESC    


(20 rows affected)

Total execution time: 00:00:00.163

ProductName,Subcategory,Category,TotalSales,PctOfTotal
Olive Oil - Extra Virgin ORG,Oils,Grocery,45039.15,0.034412
Coffee - The Bean Whole,The Bean Whole,Local Love,35283.34,0.026958
CRU Kombucha,Kombucha,Local Love,28571.28,0.021830
Raw Cashews ORG,Nuts,Grocery,23603.48,0.018034
Coffee - Hugo Coffee,Hugo Coffee,Local Love,23451.72,0.017918
Maple Syrup ORG,Sweeteners,Grocery,22980.91,0.017558
Raw Almonds ORG,Nuts,Grocery,20475.71,0.015644
Eggs ORG,Dairy,Grocery,20091.05,0.015350
Sapadilla Hand Soap,Body Cleansing,Body Care,19389.01,0.014814
Milk Honey Yogurt,Milk Honey Yogurt,Local Love,18611.99,0.014220


#### Top Selling Products by Pounds Sold

In [33]:
SELECT 
    sc.Subcategory,
    c.Category,
    p.ProductID,
    p.ProductName,
    SUM(l.PoundsSold) AS PoundsSold

FROM SalesOrderLines AS l
    JOIN DimProduct AS p
    ON l.ProductID = p.ProductID
    JOIN DimSubcategory AS sc
    ON p.SubcategoryID = sc.SubcategoryID
    JOIN DimCategory AS c
    ON sc.CategoryID = c.CategoryID

WHERE p.PricingType = 'PerPound'

GROUP BY sc.Subcategory, c.Category, p.ProductID, p.ProductName
ORDER BY PoundsSold DESC

(335 rows affected)

Total execution time: 00:00:00.129

Subcategory,Category,ProductID,ProductName,PoundsSold
Vinegars,Grocery,5,White Vinegar - 10% acidity (Cleaning),5800.2368
Laundry Cleaners,Cleaning Supply,88,EcosPro Laundry Detergent - Free & Clear,5048.9640
Seeds And Grains,Grocery,56,Brown Rice ORG,3104.8088
Nuts,Grocery,200,Almond Butter / Diced Almonds,2356.2550
Seeds And Grains,Grocery,36,"White Rice, Long Grain CONV",2305.8752
Seeds And Grains,Grocery,105,Quinoa ORG,1688.5798
Beans And Legumes,Grocery,80,Black Turtle Beans ORG,1651.7766
Cereals,Grocery,734,French Vanilla Granola,1591.4087
Laundry Cleaners,Cleaning Supply,4,Dropps Laundry Pods,1260.2290
Salts,Grocery,117,Real Salt - Sea Salt,937.8087


## Average Order Value

#### Average Order Value by Month and Moving 3 Month Average (Includes Retail and Wholesale)

In [12]:
SELECT 
    EOMONTH(o.OrderDate) AS Month,
    COUNT(DISTINCT o.OrderID) AS OrderCount,
    SUM(l.SaleAmount) AS Amount,
    SUM(l.SaleAmount) / COUNT(DISTINCT o.OrderID) AS AvgOrderValue,
    AVG(SUM(l.SaleAmount)/COUNT(DISTINCT o.OrderID)) OVER(
        ORDER BY EOMONTH(o.OrderDate) -- Order the records by transaction month, ensuring the average is calculated chronologically
        ROWS BETWEEN 2 PRECEDING and CURRENT ROW -- specifies the range of rows to include in the average as 2 preceeding and current row
    ) AS MovingThreeMoAvgAOV
FROM
    SalesOrderLines AS l
    JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID

GROUP BY EOMONTH(o.OrderDate)
ORDER BY EOMONTH(o.OrderDate)

(35 rows affected)

Total execution time: 00:00:00.153

Month,OrderCount,Amount,AvgOrderValue,MovingThreeMoAvgAOV
2019-02-28,749,14964.79,19.979692,19.979692
2019-03-31,1003,20949.45,20.886789,20.433240
2019-04-30,899,17287.15,19.229310,20.031930
2019-05-31,1184,24424.98,20.629206,20.248435
2019-06-30,1137,27454.37,24.146323,21.334946
2019-07-31,1120,27423.94,24.485660,23.087063
2019-08-31,1095,26306.96,24.024621,24.218868
2019-09-30,1109,28842.81,26.007944,24.839408
2019-10-31,1135,29368.16,25.875030,25.302531
2019-11-30,1153,29010.85,25.161188,25.681387


#### Retail AOV and Three Month Moving Average


In [47]:
SELECT 
    EOMONTH(o.OrderDate) AS Month,
    COUNT(DISTINCT o.OrderID) AS OrderCount,
    SUM(l.SaleAmount) AS Amount,
    SUM(l.SaleAmount) / COUNT(DISTINCT o.OrderID) AS AvgOrderValue,
    AVG(SUM(l.SaleAmount)/COUNT(DISTINCT o.OrderID)) OVER(
        ORDER BY EOMONTH(o.OrderDate) -- Order the records by transaction month, ensuring the average is calculated chronologically
        ROWS BETWEEN 2 PRECEDING and CURRENT ROW -- specifies the range of rows to include in the average as 2 preceeding and current row
    ) AS MovingThreeMoAvgAOV
FROM
    SalesOrderLines AS l
    JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID
    JOIN dimCustomers AS c
    ON o.CustomerID = c.CustomerID

WHERE c.CustomerType = 'Retail'

GROUP BY EOMONTH(o.OrderDate)
ORDER BY EOMONTH(o.OrderDate)

(35 rows affected)

Total execution time: 00:00:00.204

Month,OrderCount,Amount,AvgOrderValue,MovingThreeMoAvgAOV
2019-02-28,749,14964.79,19.979692,19.979692
2019-03-31,1000,20094.08,20.094080,20.036886
2019-04-30,896,16817.66,18.769709,19.614493
2019-05-31,1177,23642.95,20.087468,19.650419
2019-06-30,1126,24337.25,21.613898,20.157025
2019-07-31,1107,23399.75,21.137985,20.946450
2019-08-31,1088,24639.41,22.646516,21.799466
2019-09-30,1096,25514.16,23.279343,22.354614
2019-10-31,1126,27212.48,24.167388,23.364415
2019-11-30,1145,27021.07,23.599187,23.681972


## Customer Analysis

#### Compare Retail and Wholesale Sales Statistics


In [35]:
SELECT
    c.CustomerType,
    SUM(SaleAmount) AS Sales,
    SUM(SaleAmount)/
    (SELECT 
        SUM(SaleAmount)
        FROM SalesOrderLines) AS PctOfSales, --Use a subquery to calculate total sales and percent of total sales
    COUNT(DISTINCT o.OrderID) As TransactionCount,
    SUM(SaleAmount) / COUNT(DISTINCT o.OrderID) AS AvgOrderValue,
    COUNT(DISTINCT o.CustomerID) AS UniqueCustomers

FROM SalesOrderLines AS l
    JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID   
    JOIN dimCustomers AS c
    ON o.CustomerID = c.CustomerID

GROUP BY c.CustomerType

(2 rows affected)

Total execution time: 00:00:00.174

CustomerType,Sales,PctOfSales,TransactionCount,AvgOrderValue,UniqueCustomers
Retail,1251542.46,0.956252,41220,30.362505,3917
Wholesale,57256.92,0.043747,230,248.943130,15


#### Retail Sales: Known Customers vs Anonymous Sales - CASE Statement and CTE to group by CASE Statement

Sales Composition (% of Total), AOV, Unique Customers, Order Count

In [36]:
-- Create a CTE of Sales Transactions, adding a Customer Type using a CASE statement
WITH CustomerSales AS(
    SELECT
        o.CustomerID,
        
        CASE 
            WHEN o.CustomerID IS NULL THEN 'Blank'
            WHEN o.CustomerID = 'Unknown' THEN 'Anonymous'
            ELSE 'KnownCustomers'
            END As CustomerType,
        
        l.SaleAmount,
        o.OrderID AS OrderID
    
    FROM SalesOrderLines AS l
        JOIN SalesOrders AS o
        ON l.OrderID = o.OrderID   
        JOIN dimCustomers AS c
        ON o.CustomerID = c.CustomerID

    WHERE c.CustomerType = 'Retail'
)

-- Create a query subtotaling by the newly assigned Customer Type
SELECT
    CustomerType,
    SUM(SaleAmount) AS Sales,
    SUM(SaleAmount)/
    (SELECT 
        SUM(SaleAmount)
        FROM CustomerSales) AS PctOfSales, --Use a subquery to calculate total sales and percent of total sales
    COUNT(DISTINCT OrderID) As OrderCount,
    SUM(SaleAmount) / COUNT(DISTINCT OrderID) AS AvgOrderValue,
    COUNT(DISTINCT CustomerID) AS UniqueCustomers
FROM CustomerSales
GROUP BY CustomerType

(2 rows affected)

Total execution time: 00:00:00.237

CustomerType,Sales,PctOfSales,OrderCount,AvgOrderValue,UniqueCustomers
KnownCustomers,791712.11,0.632589,22577,35.067197,3916
Anonymous,459830.35,0.367410,18643,24.665040,1


#### Mean Purchase Frequency for All Known Retail Customers (with CustomerID)

In [37]:
--Mean Purchase Frequency for All Known Customers (with Customer ID)

-- Create a CTE grouped by Customer ID, that calculates the active period (days between first and last purchase), 
-- and divides that by the number of transactions in the period to determine the purchase frequency for each known customer ID
WITH PurchaseFrequency AS(
    SELECT
    o.CustomerID,
    COUNT(DISTINCT o.OrderID) TransactionCount,
    DATEDIFF(Day,MIN(o.OrderDate),MAX(o.OrderDate)) AS ActivePeriod,
    CASE 
        WHEN DATEDIFF(Day,MIN(o.OrderDate),MAX(o.OrderDate)) = 0 THEN 0
            ELSE DATEDIFF(Day,MIN(o.OrderDate),MAX(o.OrderDate))/ COUNT(DISTINCT o.OrderID) 
        END AS PurchaseFreqencyDays

FROM SalesOrderLines AS l
        JOIN SalesOrders AS o
        ON l.OrderID = o.OrderID   
        JOIN dimCustomers AS c
        ON o.CustomerID = c.CustomerID
WHERE o.CustomerID <> 'Unknown' 
AND  c.CustomerType = 'Retail'
GROUP BY o.CustomerID
)

-- From the CTE above, average the Purchase Frequency to determine the average for all known customers
SELECT 
    AVG(PurchaseFreqencyDays) AS MeanPurchaseFrequencyDays,
    AVG(TransactionCount) AS MeanTransactionCount
FROM PurchaseFrequency

(1 row affected)

Total execution time: 00:00:00.158

MeanPurchaseFrequencyDays,MeanTransactionCount
33,5


## Day of Week Sales

#### Avg Transaction Count, Daily Sales and AOV by Day of the Week - Retails Sales

In [39]:
-- Create a CTE of Daily Sales
WITH Daily AS(
    SELECT
        o.OrderDate,
        DATENAME(WEEKDAY,o.OrderDate) AS DayofWeek,
        DATEPART(WEEKDAY, o.OrderDate) AS DayNumber,
        COUNT(DISTINCT o.OrderID) AS DailyTransactions,
        SUM(l.SaleAmount) AS DailySales,
        SUM(l.SaleAmount) / COUNT(DISTINCT o.OrderID) AS AvgOrderValue
    FROM
        SalesOrderLines AS l
        JOIN SalesOrders AS o
        ON l.OrderID = o.OrderID
        JOIN DimCustomers AS c
        ON o.CustomerID = c.CustomerID

    WHERE c.CustomerType = 'Retail'
    
    GROUP BY o.OrderDate 
)

-- Summarize Sales Data by Day of Week
SELECT
    DayofWeek,
    Sum(DailyTransactions) AS TotalTransactionCount,
    AVG(DailyTransactions) AS AvgDailyTransactions,
    Sum(DailySales) AS TotalSales,
    Avg(DailySales) AS AvgDailySales,
    SUM(DailySales) / Sum(DailyTransactions) AS AvgOrderValue
FROM Daily
GROUP BY DayofWeek, DayNumber
ORDER BY DayNumber

(7 rows affected)

Total execution time: 00:00:00.159

DayofWeek,TotalTransactionCount,AvgDailyTransactions,TotalSales,AvgDailySales,AvgOrderValue
Sunday,5941,40,189939.19,1292.103333,31.970912
Monday,5588,38,165201.91,1139.323517,29.563691
Tuesday,5228,35,154732.23,1038.471342,29.596830
Wednesday,5020,33,145338.16,982.014594,28.951824
Thursday,4655,33,134008.38,964.089064,28.788051
Friday,5616,38,175745.27,1203.734726,31.293673
Saturday,9172,61,286577.32,1923.337718,31.244801
